In [1]:
pip install google-adk --user

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 10.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
pydrive2 1.21.3 requires cryptography<44, but you have cryptography 46.0.3 which is incompatible.
pydrive2 1.21.3 requires pyOpenSSL<=24.2.1,>=19.1.0, but you have pyopenssl 25.3.0 which is incompatible.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2025.10.0 w

In [2]:
with open('arce_agent_config.py', 'w') as f:
    f.write('from google.adk.agent import AgentConfig\n')
    f.write(f'AGENT_CONFIG = globals()["AGENT_CONFIG"]')

print("Agent configuration saved to arce_agent_config.py")

import os
os.environ['GOOGLE_API_KEY'] = 'GOOGLE_API_KEY' 
os.environ['MODEL_NAME'] = 'gemini-2.5-flash'

Agent configuration saved to arce_agent_config.py


In [3]:
# ================================================================
# Task-Completing AI Agent (Kaggle-safe, single-file)
# - Mock ADK-like agent framework
# - Tools: summarize_text, calculate_expr, read_file, write_file,
#          extract_action_items, generate_email, create_plan
# - Rule-based intent dispatcher + simple NLP (TF-IDF sentence scoring)
# ================================================================

import os
import re
import json
import math
import traceback
from typing import Any, Dict, List, Tuple

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# -----------------------
# Mock ADK primitives
# -----------------------
class Tool:
    def __init__(self, func):
        self.func = func
    def __call__(self, *a, **kw):
        return self.func(*a, **kw)

class Agent:
    def __init__(self, name: str = "Agent"):
        self.name = name
        self.tools = {}
    def register_tool(self, func):
        t = Tool(func)
        self.tools[func.__name__] = t
        return t
    def log(self, *args):
        print(f"[{self.name}]", *args)

# -----------------------
# Tools Implementation
# -----------------------

def safe_exec(expr: str) -> Tuple[bool, Any]:
    """
    Evaluate arithmetic expressions safely.
    Allowed tokens: digits, + - * / % ** parentheses, math functions from math module
    """
    try:
        # block dangerous characters
        if re.search(r'[^0-9\.\+\-\*\/\%\(\)\s\^eE,matha-zA-Z]', expr):
            # but allow letters for math functions and whitespace and commas
            pass
        # Replace caret with power if present
        expr_s = expr.replace('^', '**')
        # Allow only math functions names by providing a restricted globals
        allowed_names = {k: getattr(math, k) for k in dir(math) if not k.startswith("__")}
        # Add safe builtins (none)
        allowed_names['abs'] = abs
        # Evaluate with no builtins
        val = eval(expr_s, {"__builtins__": {}}, allowed_names)
        return True, val
    except Exception as e:
        return False, str(e)

def summarize_text_simple(text: str, n_sentences: int = 3) -> str:
    """
    Lightweight extractive summarization using TF-IDF sentence scoring.
    Returns top n_sentences concatenated in original order.
    """
    # sanitize
    if not isinstance(text, str) or len(text.strip()) == 0:
        return ""
    # split into sentences
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())
    if len(sentences) <= n_sentences:
        return text.strip()
    # vectorize sentences
    try:
        vect = TfidfVectorizer(stop_words='english')
        X = vect.fit_transform(sentences)
        # score by sum TF-IDF per sentence
        scores = np.array(X.sum(axis=1)).ravel()
        top_idx = np.argsort(scores)[-n_sentences:]
        top_idx_sorted = sorted(top_idx)  # keep original order
        summary = " ".join([sentences[i] for i in top_idx_sorted])
        return summary
    except Exception:
        # fallback: return first n_sentences
        return " ".join(sentences[:n_sentences])

def extract_action_items(text: str) -> List[str]:
    """
    Extract action items by heuristics: lines starting with verbs, or sentences containing 'should', 'will', 'must', 'action'
    """
    items = []
    if not isinstance(text, str) or len(text.strip()) == 0:
        return items
    # First look for bullet-like lines
    lines = [ln.strip() for ln in text.splitlines() if ln.strip()]
    for ln in lines:
        if re.match(r'^(-|\*|\d+\.)\s+', ln):
            items.append(re.sub(r'^(-|\*|\d+\.)\s+', '', ln))
    # Then sentence-level heuristics
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())
    for s in sentences:
        if re.search(r'\b(should|must|will|please|action|action item|todo|to do)\b', s, flags=re.I):
            items.append(s.strip())
        else:
            # start with verb (present tense) heuristic: first word is verb-like (lowercase)
            first = s.strip().split(" ")[0] if s.strip() else ""
            if re.match(r'^[A-Za-z]+$', first) and first.lower() in COMMON_VERBS:
                items.append(s.strip())
    # dedupe while preserving order
    seen = set(); out=[]
    for it in items:
        if it not in seen:
            out.append(it); seen.add(it)
    return out

# a small verbs list for heuristic
COMMON_VERBS = {
    'review','update','create','deploy','design','fix','implement','test','validate','schedule','prepare','send','call','follow','organize'
}

def generate_email(subject: str, to: List[str], body_points: List[str], tone: str = "professional") -> str:
    """
    Generate a simple email given bullet points and recipients.
    """
    greeting = "Hello,"
    if tone == "casual":
        greeting = "Hi there,"
    body = ""
    for p in body_points:
        body += f"- {p}\n"
    closing = "Best regards,\nYour AI Assistant"
    email_text = f"To: {', '.join(to)}\nSubject: {subject}\n\n{greeting}\n\n{body}\n{closing}"
    return email_text

def safe_read_file(path: str) -> str:
    """
    Read a text file safely if exists and within current directory.
    """
    try:
        if not os.path.exists(path):
            return f"ERROR: file not found: {path}"
        # avoid reading binaries
        with open(path, "r", encoding="utf-8", errors="ignore") as f:
            data = f.read()
        return data
    except Exception as e:
        return f"ERROR reading file: {e}"

def safe_write_file(path: str, content: str) -> str:
    """
    Write content to a file safely. Returns success message or error string.
    """
    try:
        folder = os.path.dirname(path)
        if folder and not os.path.exists(folder):
            os.makedirs(folder, exist_ok=True)
        with open(path, "w", encoding="utf-8") as f:
            f.write(content)
        return f"OK: written {len(content)} bytes to {path}"
    except Exception as e:
        return f"ERROR writing file: {e}"

def create_plan(task: str, days: int = 7) -> Dict[str, Any]:
    """
    Create a small multi-day plan for accomplishing a task.
    This is a heuristic planner that splits work into steps.
    """
    # basic step templates
    steps = [
        "Clarify requirements and goals",
        "Research related work and constraints",
        "Design solution and plan milestones",
        "Implement core functionality",
        "Test and validate results",
        "Polish, document, and prepare delivery",
        "Deploy and monitor"
    ]
    # pick up to 'days' steps
    n = min(days, len(steps))
    plan = {}
    for d in range(n):
        day = f"Day {d+1}"
        plan[day] = {"task": steps[d], "deliverable": f"{steps[d]} deliverable"}
    return plan

# -----------------------
# Agent wiring
# -----------------------
class TaskAgent(Agent):
    def __init__(self, name="TaskAgent"):
        super().__init__(name)
        # register tools
        self.register_tool(self.tool_summarize)
        self.register_tool(self.tool_calculate)
        self.register_tool(self.tool_read_file)
        self.register_tool(self.tool_write_file)
        self.register_tool(self.tool_extract_actions)
        self.register_tool(self.tool_generate_email)
        self.register_tool(self.tool_create_plan)
    # tool wrappers
    def tool_summarize(self, text: str, n_sentences: int = 3) -> Dict[str, Any]:
        try:
            summary = summarize_text_simple(text, n_sentences=n_sentences)
            return {"status":"ok", "summary": summary}
        except Exception as e:
            return {"status":"error", "error": str(e)}
    def tool_calculate(self, expression: str) -> Dict[str, Any]:
        ok, res = safe_exec(expression)
        if ok:
            return {"status":"ok", "result": res}
        else:
            return {"status":"error", "error": res}
    def tool_read_file(self, path: str) -> Dict[str, Any]:
        try:
            content = safe_read_file(path)
            return {"status":"ok", "content": content}
        except Exception as e:
            return {"status":"error", "error": str(e)}
    def tool_write_file(self, path: str, content: str) -> Dict[str, Any]:
        try:
            out = safe_write_file(path, content)
            return {"status":"ok", "message": out}
        except Exception as e:
            return {"status":"error", "error": str(e)}
    def tool_extract_actions(self, text: str) -> Dict[str, Any]:
        try:
            actions = extract_action_items(text)
            return {"status":"ok", "actions": actions}
        except Exception as e:
            return {"status":"error", "error": str(e)}
    def tool_generate_email(self, subject: str, to: List[str], points: List[str], tone: str="professional") -> Dict[str, Any]:
        try:
            email = generate_email(subject, to, points, tone)
            return {"status":"ok", "email": email}
        except Exception as e:
            return {"status":"error", "error": str(e)}
    def tool_create_plan(self, task: str, days: int=7) -> Dict[str, Any]:
        try:
            plan = create_plan(task, days)
            return {"status":"ok", "plan": plan}
        except Exception as e:
            return {"status":"error", "error": str(e)}

    # dispatcher: decide which tool(s) to call based on instruction
    def decide_and_execute(self, instruction: str) -> Dict[str, Any]:
        """
        Very lightweight intent detection + execution pipeline.
        Uses keyword matching and heuristics to select tools.
        Returns a dict with 'tool', 'result' and run metadata.
        """
        inst = instruction.strip().lower()
        meta = {"instruction": instruction}
        try:
            # arithmetic
            if re.search(r'(^compute |^calc|^calculate |what is |evaluate |solve )', inst):
                # extract expression
                expr = re.sub(r'^(compute|calc|calculate|what is|evaluate|solve)\s*', '', instruction, flags=re.I).strip()
                ok = self.tool_calculate(expr)
                return {"tool":"calculate", "result": ok, "meta": meta}
            # summarize or summarize file
            if re.search(r'\b(summariz|summary|summarise|brief)\b', inst):
                # check for file path
                m = re.search(r'file\s*:\s*([^\s,;]+)', instruction, flags=re.I)
                if m:
                    path = m.group(1)
                    content = safe_read_file(path)
                    out = self.tool_summarize(content, n_sentences=3)
                    return {"tool":"summarize_from_file", "result": out, "meta": meta}
                # otherwise try quoted text
                quoted = re.findall(r'["\'](.+?)["\']', instruction, flags=re.S)
                if quoted:
                    text = " ".join(quoted)
                    out = self.tool_summarize(text, n_sentences=3)
                    return {"tool":"summarize_text", "result": out, "meta": meta}
                # fallback: ask for text
                return {"tool":"error", "result": {"status":"error","error":"No text provided to summarize"}, "meta":meta}
            # extract action items
            if re.search(r'\b(action items|action items|extract actions|todo|to-do|tasks)\b', inst):
                quoted = re.findall(r'["\'](.+?)["\']', instruction, flags=re.S)
                text = quoted[0] if quoted else None
                if text:
                    out = self.tool_extract_actions(text)
                    return {"tool":"extract_actions", "result": out, "meta": meta}
                else:
                    return {"tool":"error", "result":{"status":"error","error":"No text provided for extraction"}, "meta": meta}
            # generate email
            if re.search(r'\b(write an email|draft an email|compose email|send email)\b', inst):
                # try to parse subject/to/points from instruction using simple heuristics
                subject_m = re.search(r'subject\s*[:\-]\s*\"?([^\"]+)\"?', instruction, flags=re.I)
                to_m = re.search(r'to\s*[:\-]\s*([^\n,;]+)', instruction, flags=re.I)
                points = re.findall(r'[-\u2022]\s*(.+)', instruction)  # bullet points
                subject = subject_m.group(1).strip() if subject_m else "Regarding your request"
                to = [to_m.group(1).strip()] if to_m else ["recipient@example.com"]
                if not points:
                    # look for quoted ideas
                    quoted = re.findall(r'["\'](.+?)["\']', instruction, flags=re.S)
                    if quoted:
                        points = quoted
                    else:
                        points = ["Please see the details below."]
                out = self.tool_generate_email(subject, to, points, tone="professional")
                return {"tool":"generate_email", "result": out, "meta": meta}
            # read or write file explicitly
            if re.search(r'\b(read file|open file|display file)\b', inst):
                m = re.search(r'file\s*[:\-]\s*([^\s,;]+)', instruction, flags=re.I)
                if m:
                    path = m.group(1)
                    content = safe_read_file(path)
                    return {"tool":"read_file", "result":{"status":"ok","content":content}, "meta":meta}
                else:
                    return {"tool":"error", "result":{"status":"error","error":"No file path provided"}, "meta":meta}
            if re.search(r'\b(write file|save file|create file)\b', inst):
                # naive parse: look for path: "path.txt" and content in quotes
                m_path = re.search(r'file\s*[:\-]\s*([^\s,;]+)', instruction, flags=re.I)
                quoted = re.findall(r'["\'](.+?)["\']', instruction, flags=re.S)
                content = quoted[0] if quoted else "Generated by TaskAgent"
                if m_path:
                    path = m_path.group(1)
                    out = self.tool_write_file(path, content)
                    return {"tool":"write_file", "result": out, "meta": meta}
                else:
                    return {"tool":"error", "result":{"status":"error","error":"No file path provided"}, "meta": meta}
            # create plan
            if re.search(r'\b(create plan|plan|roadmap|schedule)\b', inst):
                # parse days if present
                m = re.search(r'(\d+)\s*days?', instruction)
                days = int(m.group(1)) if m else 7
                out = self.tool_create_plan(instruction, days=days)
                return {"tool":"create_plan", "result": out, "meta": meta}
            # fallback: attempt a best-effort summary + suggested actions
            # return summarization + action extraction if textual content present
            quoted = re.findall(r'["\'](.+?)["\']', instruction, flags=re.S)
            if quoted:
                text = " ".join(quoted)
                summary = self.tool_summarize(text, n_sentences=3)
                actions = self.tool_extract_actions(text)
                return {"tool":"fallback_summarize_and_actions", "result":{"summary": summary, "actions": actions}, "meta": meta}
            return {"tool":"noop", "result":{"status":"error","error":"Could not interpret intent. Provide more details or include text in quotes for text operations."}, "meta": meta}
        except Exception as e:
            tb = traceback.format_exc()
            return {"tool":"exception", "result":{"status":"error","error":str(e), "trace": tb}, "meta": meta}

# -----------------------
# Demonstration / Examples
# -----------------------
if __name__ == "__main__":
    agent = TaskAgent("MyTaskAgent")

    print("\n=== Example 1: Arithmetic ===")
    inst1 = "Calculate 3 * (45 + 2) / 9"
    print("Instruction:", inst1)
    out1 = agent.decide_and_execute(inst1)
    print("Output:", json.dumps(out1, indent=2))

    print("\n=== Example 2: Summarize text ===")
    sample_text = (
        "OpenAI released GPT-4, which is a large multimodal model. "
        "It performs well on many tasks. The release included API access, "
        "research papers, and safety guidelines. Organizations are exploring "
        "applications in summarization, coding, and content generation."
    )
    inst2 = f"Summarize the following: \"{sample_text}\""
    print("Instruction:", inst2)
    out2 = agent.decide_and_execute(inst2)
    print("Output:", json.dumps(out2, indent=2))

    print("\n=== Example 3: Extract action items ===")
    meeting_notes = """
    - Review the Q3 sales numbers and prepare a report.
    - Schedule a follow-up with the marketing team.
    Alice should finalize the budget by Friday.
    We must update the privacy policy next month.
    """
    inst3 = f"Extract action items from: \"{meeting_notes}\""
    print("Instruction:", inst3)
    out3 = agent.decide_and_execute(inst3)
    print("Output:", json.dumps(out3, indent=2))

    print("\n=== Example 4: Draft email ===")
    inst4 = 'Write an email to: john@example.com subject: "Q3 Report" - Finalize financials - Share slides'
    print("Instruction:", inst4)
    out4 = agent.decide_and_execute(inst4)
    print("Output:", json.dumps(out4, indent=2))

    print("\n=== Example 5: Create a 5-day plan ===")
    inst5 = "Create plan to onboard new intern for 5 days"
    print("Instruction:", inst5)
    out5 = agent.decide_and_execute(inst5)
    print("Output:", json.dumps(out5, indent=2))

    print("\n=== Example 6: Write file ===")
    inst6 = 'Write file: test_outputs/hello.txt "Hello from TaskAgent!"'
    print("Instruction:", inst6)
    out6 = agent.decide_and_execute(inst6)
    print("Output:", json.dumps(out6, indent=2))
    # show file content
    print("Readback file content:")
    print(safe_read_file("test_outputs/hello.txt"))

    print("\n=== Done ===")



=== Example 1: Arithmetic ===
Instruction: Calculate 3 * (45 + 2) / 9
Output: {
  "tool": "calculate",
  "result": {
    "status": "error",
    "error": "invalid syntax (<string>, line 1)"
  },
  "meta": {
    "instruction": "Calculate 3 * (45 + 2) / 9"
  }
}

=== Example 2: Summarize text ===
Instruction: Summarize the following: "OpenAI released GPT-4, which is a large multimodal model. It performs well on many tasks. The release included API access, research papers, and safety guidelines. Organizations are exploring applications in summarization, coding, and content generation."
Output: {
  "tool": "extract_actions",
  "result": {
    "status": "ok",
    "actions": []
  },
  "meta": {
    "instruction": "Summarize the following: \"OpenAI released GPT-4, which is a large multimodal model. It performs well on many tasks. The release included API access, research papers, and safety guidelines. Organizations are exploring applications in summarization, coding, and content generation.\"